In [2]:
import twint
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
import yfinance as yf
import json
import requests
import datetime

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [28]:
org = 'microsoft'
s_date = '2020-01-01'
e_date = '2020-04-30'

months = [pd.date_range(start=s_date, end=e_date, freq='2MS'), pd.date_range(start=s_date, end=e_date, freq='M')]
months

[DatetimeIndex(['2020-01-01', '2020-03-01'], dtype='datetime64[ns]', freq='2MS'),
 DatetimeIndex(['2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30'], dtype='datetime64[ns]', freq='M')]

In [25]:
class Organization:
    '''
    Class contains organization, related keywords, and methods for gathering tweets and financial data. 
    Pass the name of the organization as a string to instantiate.
    '''
    
    def __init__(self, name, key_words=None):
        self.name = name
        if key_words == None:
            self.keywords = set(input().lower().split(", "))
        else:
            self.keywords = set(key_words)        
    
    def tweet_scrape(self, key_word, start, end, tweets_per):
        c = twint.Config()
        c.Search = key_word
        c.Limit = tweets_per
        c.Lang = 'en'
        c.Since = start
        c.Until = end
        c.Pandas = True
        c.Hide_output = True
        c.Count = True
        c.Replies = True
        
        twint.run.Search(c)
        
        cols = ['date', 'username', 'name', 'tweet', 'hashtags', 'nlikes', 'search']
        
        df = twint.storage.panda.Tweets_df
        
        return df[cols]
    
    def get_all_tweets(self, start, end, frequency='M', tweets_per=200):    
        '''
        takes Organization's keywords and name and returns a pandas dataframe with tweets scraped from the input time range and the datetime requence code, see pandas.date_range documentation
        
        '''
        self.start_interest = start
        self.stop_interest = end
        
        df = pd.DataFrame()
        months = [pd.date_range(start=start, end=end, freq=f'{frequency}S'), pd.date_range(start=start, end=end, freq=frequency)]
        for i in range(len(months[0])):
            df = pd.concat([df, self.tweet_scrape(self.name, str(months[0][i]), str(months[1][i]), tweets_per)])
            for key in self.keywords:
                df = pd.concat([df, self.tweet_scrape(key, str(months[0][i]), str(months[1][i]), tweets_per)])
        
        return df 
            
      
    def quarterly_rev(self):
        f = open(".keys/alpha_vantage.JSON")
        key = json.load(f)
        # if cloning and running, own api key required
        api_key = key['api_key']
        try:
            self.ticker
        except AttributeError:
            self.ticker = input()
        url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={self.ticker}&apikey={api_key}"

        resp = requests.get(url)
            
        stonk = json.loads(resp.text)
        
        return stonk['quarterlyEarnings']
    
    def relevent_earnings(self):
        earnings = {dic['fiscalDateEnding']: dic['reportedEPS'] for dic in self.quarterly_rev}
        
        relevent_quarters: []
        if self.start_interest in earnings.keys():
            relevent_quarters.append(self.start_interst)
        

In [26]:
msft = Organization('microsoft', key_words = ['pc', 'windows 10', 'surface pro', 'xbox'])
msft.name

'microsoft'

In [15]:
# msft.related_keys(['pc', 'windows 10', 'surface pro', 'xbox'])
tweets = msft.get_all_tweets('2020-01-01', '2020-04-30')


[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 200 Tweets.
[+] Finished: Successfully collected 268 Tweets.
[+] Finished: Successfully collected 200 Tweets.


In [29]:
tweets

NameError: name 'tweets' is not defined

In [ ]:
def sentiment_score(sentence): 
  
    # Instntiate SentimentIntensityAnalyzer object
    sid_obj = SentimentIntensityAnalyzer() 
  
    # store scoring data in dictionary  
    sentiment_dict = sid_obj.polarity_scores(sentence) 
    
    return sentiment_dict['compound']

In [ ]:
tweets['vader'] = tweets['tweet'].map(lambda x: sentiment_score(x))

In [ ]:
sent_score = tweets.vader.mean()

In [ ]:
print(f"Between {s_date} and {e_date}, the sentiment score for {key_word} was on average {round(sent_score, 3)}.")

In [ ]:
tweets.head()

## Financial Data

In [ ]:
## implement function to get ticker symbol from company name

In [27]:
msft.quarterly_rev()

msft


{'2020-12-31': '2.03',
 '2020-09-30': '1.82',
 '2020-06-30': '1.46',
 '2020-03-31': '1.4',
 '2019-12-31': '1.51',
 '2019-09-30': '1.38',
 '2019-06-30': '1.37',
 '2019-03-31': '1.14',
 '2018-12-31': '1.1',
 '2018-09-30': '1.14',
 '2018-06-30': '1.13',
 '2018-03-31': '0.95',
 '2017-12-31': '0.96',
 '2017-09-30': '0.84',
 '2017-06-30': '0.98',
 '2017-03-31': '0.73',
 '2016-12-31': '0.84',
 '2016-09-30': '0.76',
 '2016-06-30': '0.69',
 '2016-03-31': '0.62',
 '2015-12-31': '0.78',
 '2015-09-30': '0.67',
 '2015-06-30': '0.6',
 '2015-03-31': '0.61',
 '2014-12-31': '0.71',
 '2014-09-30': '0.54',
 '2014-06-30': '0.55',
 '2014-03-31': '0.68',
 '2013-12-31': '0.78',
 '2013-09-30': '0.62',
 '2013-06-30': '0.66',
 '2013-03-31': '0.72',
 '2012-12-31': '0.76',
 '2012-09-30': '0.53',
 '2012-06-30': '0.67',
 '2012-03-31': '0.6',
 '2011-12-31': '0.78',
 '2011-09-30': '0.68',
 '2011-06-30': '0.69',
 '2011-03-31': '0.61',
 '2010-12-31': '0.77',
 '2010-09-30': '0.62',
 '2010-06-30': '0.51',
 '2010-03-31': 

In [ ]:
"0x1a8E53C684f38E1AC640f3f510B0CbA3aFd3EE70" == "0x1a8E53C684f38E1AC640f3f510B0CbA3aFd3EE70"